In [2]:
import cv2
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation
import os
import dlib
import numpy as np

In [3]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
cap.set(cv2.CAP_PROP_FPS, 60)

segmentor = SelfiSegmentation()
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/91797/Downloads/shape_predictor_68_face_landmarks.dat")

fpsReader = cvzone.FPS()

while True:
    success, img = cap.read()
    imgOut = segmentor.removeBG(img, (0,0,0), threshold=0.99)
    #_, frame = cap.read()
    gray = cv2.cvtColor(imgOut, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    b, g, r = cv2.split(img)
    wb = b == 255
    wg = g == 255
    wr = r == 255

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        landmarks = predictor(gray, face)
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(imgOut, (x, y), 4, (0, 0, 255), -1)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_range = np.array([150,100,100])
    upper_range = np.array([359,255,255])
    mask = cv2.inRange(hsv, lower_range, upper_range)

    white_pixels_if_true = np.bitwise_and(wb, np.bitwise_and(wg, wr))
    img_size = r.size
    white_pixels_count = np.sum(white_pixels_if_true)
    white_area_ratio = white_pixels_count / img_size

    imgStacked = cvzone.stackImages([img, imgOut, mask],3,1)
    _, imgStacked = fpsReader.update(imgStacked, color = (0, 0, 255))
    if (white_area_ratio >= 0.00001):
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(imgStacked, 'Drowsy Alert', (50,150), font, 1, (0,255,255),2,cv2.LINE_4)
    cv2.imshow("Redness of Mouth", imgStacked)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.release()
cv2.destroyAllWindows